In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=2
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# dx = 1km; Np = 50M; Nz = 95
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc') #***
parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc') #***
res='1km'; t_res='1min_95nz'; Np_str='50e6'

In [ ]:
times=data1['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data1['xh'].values-data1['xh'][0].values >= 1)

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
########################
#READING BACK IN
def LoadFinalData(in_file):
    dict = {}
    with h5py.File(in_file, 'r') as f:
        for key in f.keys():
            dict[key] = f[key][:]
    return dict

def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
print(f"Minimum Cloudbase is: {min_all_cloudbase}\n")

dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
in_file=dir2+f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}.h5"
final_dict=LoadFinalData(in_file)


#DYNAMICALLY CREATING VARIABLES
for key, value in final_dict.items():
    globals()[key] = value

# #DYNAMICALLY PRINTING VARIABLE SIZES
# for key in final_dict:
#     print(f"{key} has {final_dict[key].shape[0]} parcels")

# PRINTING VARIABLE SIZES (ONE BY ONE)
print(f'ALL: {len(CL_ALL_out_arr)} CL parcels and {len(nonCL_ALL_out_arr)} nonCL parcels')
print(f'SHALLOW: {len(CL_SHALLOW_out_arr)} CL parcels and {len(nonCL_SHALLOW_out_arr)} nonCL parcels')
print(f'DEEP: {len(CL_DEEP_out_arr)} CL parcels and {len(nonCL_DEEP_out_arr)} nonCL parcels')
print('\n')
print(f'ALL: {len(SBZ_ALL_out_arr)} SBZ parcels and {len(nonSBZ_ALL_out_arr)} nonSBZ parcels')
print(f'SHALLOW: {len(SBZ_SHALLOW_out_arr)} SBZ parcels and {len(nonSBZ_SHALLOW_out_arr)} nonSBZ parcels')
print(f'DEEP: {len(SBZ_DEEP_out_arr)} SBZ parcels and {len(nonSBZ_DEEP_out_arr)} nonSBZ parcels')
print('\n')
print(f'ALL: {len(ColdPool_ALL_out_arr)} ColdPool parcels')
print(f'SHALLOW: {len(ColdPool_SHALLOW_out_arr)} ColdPool parcels')
print(f'DEEP: {len(ColdPool_DEEP_out_arr)} ColdPool parcels')


def apply_job_array_filter(start_job, end_job):
    # print("APPLYING JOB ARRAY")

    def job_filter(arr):
        return arr[(arr[:, 0] >= start_job) & (arr[:, 0] < end_job)]

    target_names = [
        'CL_ALL_out_arr', 'nonCL_ALL_out_arr',
        'CL_SHALLOW_out_arr', 'nonCL_SHALLOW_out_arr',
        'CL_DEEP_out_arr', 'nonCL_DEEP_out_arr',
        'SBZ_ALL_out_arr', 'nonSBZ_ALL_out_arr',
        'SBZ_SHALLOW_out_arr', 'nonSBZ_SHALLOW_out_arr',
        'SBZ_DEEP_out_arr', 'nonSBZ_DEEP_out_arr',
        'ColdPool_ALL_out_arr', 'ColdPool_SHALLOW_out_arr', 'ColdPool_DEEP_out_arr'
    ]

    for name in target_names:
        globals()[name+'_2'] = job_filter(globals()[name])

In [ ]:
##############################################
#SETUP

In [ ]:
PROCESSING=False
PROCESSING=True

In [ ]:
def MakeDictionary(**vars):
    return vars
    
def GetData1(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['W', 'QCQI']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    W, QCQI = (data_dict[k] for k in var_names)
    
    # #Making Time Matrix
    # rows, cols = A.shape[0], A.shape[1]
    # T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
    
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_VARS_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['QV','TH','TH_E','BUOYANCY','HMC']#,'QI','QR']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    QV, TH, TH_E, BUOYANCY, HMC = [data_dict[k] for k in var_names] #, QI, QR

    VARs=MakeDictionary(W=W, QCQI=QCQI, QV=QV, 
                        TH=TH, TH_E=TH_E, BUOYANCY=BUOYANCY, HMC=HMC)#, QI=QI, QR=QR)
    return VARs

def GetData2(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_VARS_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['VMF_C','VMF_G']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    VMF_C, VMF_G = [data_dict[k] for k in var_names]
    
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}.h5'
    if PROCESSING==False:
        in_file=dir2+f'lagrangian_ED_VARS_array_{res}_{t_res}_{Np_str}_ORIGINAL.h5'
        
    
    var_names = ['E_G','E_C','D_C','D_G']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    E_G, E_C, D_C, D_G = (data_dict[k] for k in var_names)

    NET_G, NET_C = E_G-D_G, E_C-D_C

    VARs=MakeDictionary(VMF_C=VMF_C, VMF_G=VMF_G, E_G=E_G, E_C=E_C, D_C=D_C, D_G=D_G, NET_G=NET_G, NET_C=NET_C)
    return VARs

def GetData3(start_job,end_job):    
    dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir3+f'lagrangian_Combined_ED_VARS_array_{res}_{t_res}_{Np_str}.h5' 
    if PROCESSING==False:
        in_file=dir3+f'lagrangian_Combined_ED_VARS_array_{res}_{t_res}_{Np_str}_ORIGINAL.h5' 
    
    var_names = ['C_TO_G','G_TO_C']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    C_TO_G, G_TO_C = (data_dict[k] for k in var_names)

    VARs=MakeDictionary(C_TO_G=C_TO_G, G_TO_C=G_TO_C)
    return VARs

In [ ]:
################################
#DATA SETUP
################################
#*#*
# data_type="Tracked_Properties" 
data_type="Tracked_Entrainment_VMF"
# data_type="Tracked_Combined_Entrainment" 

if data_type=="Tracked_Properties":
    GetData=GetData1
    variables = ["W","QCQI","QV","TH","TH_E","BUOYANCY","HMC"]#,"QI","QR"]
elif data_type=="Tracked_Entrainment_VMF":
    GetData=GetData2
    variables = ["VMF_C","VMF_G","E_G","E_C","D_C","D_G","NET_G","NET_C"]
elif data_type=="Tracked_Combined_Entrainment":
    GetData=GetData3
    variables = ["C_TO_G","G_TO_C"]
##############################

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if Np_str=='1e6':
    num_jobs=60 #1M parcels
    num_slurm_jobs=60
if Np_str=='50e6':
    num_jobs=200 #50M parcels
    num_slurm_jobs=60
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(parcel1['xh']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust

# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type,start_job,end_job):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            data_dict = {var_name: f[var_name][:,start_job:end_job] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        data_dict = {k: in_data[k][:,start_job:end_job].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
def GetSpatialData(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['Z', 'Y', 'X', 'z']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    Z, Y, X, parcel_z = (data_dict[k] for k in var_names)
    
    # #Making Time Matrix
    # rows, cols = A.shape[0], A.shape[1]
    # T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
    
    return Z,Y,X,parcel_z

In [ ]:
############################################################################################

In [ ]:
##############################################
#MAKE PROFILES FUNCTIONS

In [ ]:
def TrackedProfile(var_data,Z,Y,X,type1,type2):
    global index_adjust
    
    out_arr=globals()[f"{type2}_{type1.upper()}_out_arr"+"_2"].copy()
    
    zhs=data1['zh'].values
    profile_array =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
    profile_array[:,2]=zhs;
    
    for row in range(out_arr.shape[0]):
        after=out_arr[row,3]
        # if np.mod(row,3000)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)

        zs=Z[ts,p-index_adjust]
        ys=Y[ts,p-index_adjust]
        xs=X[ts,p-index_adjust]
        
        vars=var_data[ts,p-index_adjust]
        np.add.at(profile_array[:, 0], zs, vars)
        np.add.at(profile_array[:, 1], zs, 1)
    return profile_array

In [ ]:
def TrackedProfile_Simultaneous(VARs,Z,Y,X,type1,type2):
    global index_adjust

    #READING IN TRACKED PARCEL LISTS
    out_arr=globals()[f"{type2}_{type1.upper()}_out_arr"+"_2"].copy()

    #MAKE PROFILE_ARRAYS TO HOLD THE RESULT
    profile_dict = {}
    zhs = data1['zh'].values  # outside the loop for efficiency
    for var_name in VARs:
        key = f"{type2}_{type1.upper()}_profile_array_{var_name}"
        profile_array = np.zeros((len(zhs), 3))
        profile_array[:, 2] = zhs
        profile_dict[key] = profile_array

    #CALCULATING
    for row in range(out_arr.shape[0]):
        after=out_arr[row,3]
        # if np.mod(row,3000)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)

        zs=Z[ts,p-index_adjust]
        ys=Y[ts,p-index_adjust]
        xs=X[ts,p-index_adjust]

        for var_name, var_data in VARs.items():
            key = f"{type2}_{type1.upper()}_profile_array_{var_name}"
            vars=var_data[ts,p-index_adjust]
            np.add.at(profile_dict[key][:, 0], zs, vars)
            np.add.at(profile_dict[key][:, 1], zs, 1)
    return profile_dict

In [ ]:
def RunCalculation(VARs, Z,Y,X):
    #VARs is a dictionary created by the function MakeDictionary
    Dictionary = {}

    type1_options = ['all', 'shallow', 'deep']
    type2_options = ['CL', 'nonCL', 'SBZ', 'nonSBZ', 'ColdPool']

    for t1 in type1_options:
        for t2 in type2_options:
            for var_name, VAR in VARs.items():
                key = f"{t2}_{t1.upper()}_profile_array_{var_name}"
                Dictionary[key] = TrackedProfile(VAR, Z,Y,X, t1, t2)
    return Dictionary

In [ ]:
def RunCalculation_Simultaneous(VARs, Z,Y,X):
    #VARs is a dictionary created by the function MakeDictionary
    Dictionary = {}

    type1_options = ['all', 'shallow', 'deep']
    type2_options = ['CL', 'nonCL', 'SBZ', 'nonSBZ', 'ColdPool']

    for t1 in type1_options:
        for t2 in type2_options:
            Single_Dictionary=TrackedProfile_Simultaneous(VARs, Z,Y,X, t1, t2)
            Dictionary.update(Single_Dictionary)
    return Dictionary

In [ ]:
def SaveProfile(Dictionary,data_type):
    # print("Saving all profiles...")

    # Construct the output file path
    dir2 = dir + 'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file = dir2 + f"{data_type}_" + f"tracked_profiles_{res}_{t_res}_{Np_str}"
    if PROCESSING==False:
        output_file+='_ORIGINAL'    
    output_file += f"_{job_id}.h5"

    # Write the entire dictionary to HDF5
    with h5py.File(output_file, "w") as h5f:
        for key, profile_data in Dictionary.items():
            h5f.create_dataset(key, data=profile_data)
            
    # print("Done saving.\n")

In [ ]:
##############################################
#RUNNING

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array
print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

    #SLICING DATA
    parcel=parcel1.isel(xh=slice(start_job,end_job))
    apply_job_array_filter(start_job, end_job)

    #GETTING DATA AND PUTTING IN A DICTIONARY
    [Z,Y,X,parcel_z] = GetSpatialData(start_job,end_job) 
    VARs=GetData(start_job,end_job)

    #CALCULATING AND SAVING
    # Dictionary=RunCalculation(VARs, Z,Y,X) #VERSION1: EACH VARIABLE SEPERATELY
    Dictionary=RunCalculation_Simultaneous(VARs, Z,Y,X) #VERSION2: EACH VARIABLE SIMULTANEOUSLY (RECOMMENDED)
    SaveProfile(Dictionary,data_type)

    #check_memory(globals())
    del VARs, Dictionary

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOBARRAY IS RUNNING
# recombine=True

In [ ]:
def Recombine(type1,type2,num_jobs):
    global variables
    print(f'recombining {type1}_{type2}')
    if type2==None:
        type2s = [f"{type1}"]
    else:
        type2s = [f"{type1}",f"{type2}"]
    types = ["ALL", "SHALLOW", "DEEP"]
    
    vars_list = []
    SE_list = []

    for t2 in type2s:
        for t in types:
            for var in variables:
                vars_list.append(f"{t2}_{t}_profile_array_{var}")

    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    #MAKING OUTPUT FILE PATH
    if type2==None:
        output_file=dir3 + f"{data_type}_" + f"{type1}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
        if PROCESSING==False:
            output_file=dir3 + f"{data_type}_" + f"{type1}_tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL.h5"
    else:
        output_file=dir3 + f"{data_type}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
        if PROCESSING==False:
            output_file=dir3 + f"{data_type}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL.h5"

    
    #MAKING PROFILES DICTIONARY
    zhs = data1['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 
    
    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,20)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=dir2 + f"{data_type}_" + f"tracked_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5"
        if PROCESSING==False:
            input_file=dir2 + f"{data_type}_" + f"tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL_{job_id}.h5"
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
if recombine==True:
    Recombine(type1='CL',type2='nonCL',num_jobs=num_jobs)
    Recombine(type1='SBZ',type2='nonSBZ',num_jobs=num_jobs)
    Recombine(type1='ColdPool',type2=None,num_jobs=num_jobs)

In [ ]:
####################################################################################

In [ ]:
##############################################
#MAKE PROFILES STANDARD ERROR FUNCTIONS

In [ ]:
def GetData_SE():
    #CL NONCL
    type1='CL';type2='nonCL'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    filePath=dir3 + f"{data_type}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    with h5py.File(filePath, 'r') as h5f:
        for key in h5f.keys():
            globals()[key] = h5f[key][:]
            # print(key)

    #SBZ NONSBZ
    type1='SBZ';type2='nonSBZ'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    filePath=dir3 + f"{data_type}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    with h5py.File(filePath, 'r') as h5f:
        for key in h5f.keys():
            globals()[key] = h5f[key][:]
            # print(key)

    #COLDPOOL
    type1='ColdPool'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    filePath=dir3 + f"{data_type}_" + f"{type1}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    with h5py.File(filePath, 'r') as h5f:
        for key in h5f.keys():
            globals()[key] = h5f[key][:]
            # print(key)

In [ ]:
def TrackedProfile_SE(profile_data,var_data, Z,Y,X,type1,type2):  
    global index_adjust
    global test #TESTING
    
    #READING IN TRACKED PARCEL LISTS
    out_arr=globals()[f"{type2}_{type1.upper()}_out_arr"+"_2"].copy() 

    #MAKE PROFILE_ARRAYS TO HOLD THE RESULT
    zhs=data1['zh'].values
    profile_array =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
    profile_array[:,2]=zhs;

    #CALCULATING
    # test=[] #TESTING
    for row in range(out_arr.shape[0]):
        after=out_arr[row,3]
        # if np.mod(row,3000)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)
        
        zs=Z[ts,p-index_adjust]
        ys=Y[ts,p-index_adjust]
        xs=X[ts,p-index_adjust]
        
        vars=var_data[ts,p-index_adjust]
        mean_mu=profile_data[zs,0]/profile_data[zs,1]
        
        np.add.at(profile_array[:, 0], zs,  (vars-mean_mu)**2) #SUMMING UP THE SQUARES
        np.add.at(profile_array[:, 1], zs,  1) #SUMMING UP THE SQUARES
    return profile_array

In [ ]:
def TrackedProfile_SE_Simultaneous(PROFILEs,VARs, Z,Y,X,type1,type2):  
    global index_adjust
    global test #TESTING
    
    #READING IN TRACKED PARCEL LISTS
    out_arr=globals()[f"{type2}_{type1.upper()}_out_arr"+"_2"].copy()

    #MAKE PROFILE_ARRAYS TO HOLD THE RESULT
    profile_dict = {}
    zhs = data1['zh'].values  # outside the loop for efficiency
    for var_name in VARs:
        key = f"{type2}_{type1.upper()}_profile_array_SE_{var_name}"
        profile_array = np.zeros((len(zhs), 3))
        profile_array[:, 2] = zhs
        profile_dict[key] = profile_array

    #CALCULATING
    # test=[] #TESTING
    for row in range(out_arr.shape[0]):
        after=out_arr[row,3]
        # if np.mod(row,3000)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)
        
        zs=Z[ts,p-index_adjust]
        ys=Y[ts,p-index_adjust]
        xs=X[ts,p-index_adjust]

        for (var_name, var_data), (_, profile_data) in zip(VARs.items(), PROFILEs.items()):
            key = f"{type2}_{type1.upper()}_profile_array_SE_{var_name}"
            vars=var_data[ts,p-index_adjust]
            mean_mu=profile_data[zs,0]/profile_data[zs,1]
            np.add.at(profile_dict[key][:, 0], zs, (vars-mean_mu)**2)
            np.add.at(profile_dict[key][:, 1], zs, 1)
    return profile_dict

In [ ]:
def RunCalculation_SE(VARs, Z,Y,X):
    #VARs is a dictionary created by the function MakeDictionary
    Dictionary = {}

    type1_options = ['all', 'shallow', 'deep']
    type2_options = ['CL', 'nonCL', 'SBZ', 'nonSBZ', 'ColdPool']

    for t1 in type1_options:
        for t2 in type2_options:
            for var_name, VAR in VARs.items():
                key = f"{t2}_{t1.upper()}_profile_array_SE_{var_name}"
                profile_data=globals()[f"{t2}_{t1.upper()}_profile_array_{var_name}"]
                Dictionary[key] = TrackedProfile_SE(profile_data,VAR, Z,Y,X, t1, t2)
    return Dictionary

In [ ]:
def RunCalculation_SE_Simultaneous(VARs, Z,Y,X):
    global variables
    #VARs is a dictionary created by the function MakeDictionary
    Dictionary = {}

    type1_options = ['all', 'shallow', 'deep']
    type2_options = ['CL', 'nonCL', 'SBZ', 'nonSBZ', 'ColdPool']

    for t1 in type1_options:
        for t2 in type2_options:
            for var_name, VAR in VARs.items():
                PROFILEs = {
                    f"{t2}_{t1.upper()}_profile_array_{var_name}": globals()[f"{t2}_{t1.upper()}_profile_array_{var_name}"]
                    for var_name in variables
                }
            Single_Dictionary=TrackedProfile_SE_Simultaneous(PROFILEs,VARs, Z,Y,X, t1, t2)
            Dictionary.update(Single_Dictionary)
    return Dictionary

In [ ]:
def SaveProfile_SE(Dictionary,data_type):
    # print("Saving all profiles...")

    # Construct the output file path
    dir2 = dir + 'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file = dir2 + f"{data_type}_" + f"tracked_profiles_SE_{res}_{t_res}_{Np_str}"
    if PROCESSING==True:
        output_file+='_PROCESSED'
    output_file += f"_{job_id}.h5"

    # Write the entire dictionary to HDF5
    with h5py.File(output_file, "w") as h5f:
        for key, profile_data in Dictionary.items():
            h5f.create_dataset(key, data=profile_data)
            
    # print("Done saving.\n")

In [ ]:
def Recombine_SE(type1,type2,num_jobs):
    global variables
    print(f'recombining {type1}_{type2}')
    if type2==None:
        type2s = [f"{type1}"]
    else:
        type2s = [f"{type1}",f"{type2}"]
    types = ["ALL", "SHALLOW", "DEEP"]
    
    vars_list = []
    SE_list = []

    for t2 in type2s:
        for t in types:
            for var in variables:
                vars_list.append(f"{t2}_{t}_profile_array_SE_{var}")

    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    #MAKING OUTPUT FILE PATH
    if type2==None:
        output_file=dir3+f"{data_type}_" +f"{type1}_tracked_profiles_SE_{res}_{t_res}_{Np_str}.h5"
    else:
        output_file=dir3+f"{data_type}_" +f"{type1}_{type2}_tracked_profiles_SE_{res}_{t_res}_{Np_str}.h5"
    
    #MAKING PROFILES DICTIONARY
    zhs = data1['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 
    
    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,20)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=dir2+f"{data_type}_" +f"tracked_profiles_SE_{res}_{t_res}_{Np_str}_{job_id}.h5"
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
##############################################
#RUNNING FOR STANDARD ERROR
SE_CALCULATE=False
SE_CALCULATE=True

In [ ]:
###############################
#RUNNING
if SE_CALCULATE == True:
    GetData_SE()

In [ ]:
if SE_CALCULATE == True:
    num_slurm_jobs=30
    [start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=False) #if ISRUN is False, then will not run using slurm_job_array
    
    print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job)}")
    
    job_id_list=np.arange(start_slurm_job,end_slurm_job)
    for job_id in job_id_list:
        if job_id % 1 == 0: print(f'current job_id = {job_id}')
        [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
    
        #SLICING DATA
        parcel=parcel1.isel(xh=slice(start_job,end_job))
        apply_job_array_filter(start_job, end_job)
    
        #GETTING DATA AND PUTTING IN A DICTIONARY
        [Z,Y,X,parcel_z] = GetSpatialData(start_job,end_job) 
        VARs=GetData(start_job,end_job)
    
        #CALCULATING AND SAVING
        # Dictionary=RunCalculation_SE(VARs, Z,Y,X) #VERSION1: EACH VARIABLE SEPERATELY
        Dictionary=RunCalculation_SE_Simultaneous(VARs, Z,Y,X) #VERSION2: EACH VARIABLE SIMULTANEOUSLY (RECOMMENDED)
        SaveProfile_SE(Dictionary,data_type)
    
        #check_memory(globals())
        # del VARs, Dictionary

In [ ]:
#############################################
#RECOMBINING
recombine=False #KEEP FALSE WHEN JOB_ARRAY IS RUNNING
recombine=True

In [ ]:
if recombine==True:
    Recombine_SE(type1='CL',type2='nonCL',num_jobs=num_jobs)
    Recombine_SE(type1='SBZ',type2='nonSBZ',num_jobs=num_jobs)
    Recombine_SE(type1='ColdPool',type2=None,num_jobs=num_jobs)